In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


0% [Working]
            
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease

0% [Connecting to archive.ubuntu.com] [Connecting to security.ubuntu.com (185.1
                                                                               
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease

0% [Connecting to archive.ubuntu.com (185.125.190.36)] [Waiting for headers] [W
                                                                               
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:9 http://ppa.lau

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [ ]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView("home_sales")

# Verify that the view has been created by running a simple query
result = spark.sql("SELECT * FROM home_sales LIMIT 5")
result.show()

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

query = """
SELECT 
    YEAR(date) AS year_sold, 
    ROUND(AVG(price), 2) AS avg_price
FROM 
    home_sales
WHERE 
    bedrooms = 4
GROUP BY 
    YEAR(date)
ORDER BY 
    year_sold
"""

# Execute the query
result = spark.sql(query)

# Display the results
result.show(

In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

query = """
SELECT 
    YEAR(date_built) AS year_built, 
    ROUND(AVG(price), 2) AS avg_price
FROM 
    home_sales
WHERE 
    bedrooms = 3 AND bathrooms = 3
GROUP BY 
    YEAR(date_built)
ORDER BY 
    year_built
"""

# Execute the query
result = spark.sql(query)

# Display the results
result.show()

In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

query = """
SELECT 
    YEAR(date_built) AS year_built, 
    ROUND(AVG(price), 2) AS avg_price
FROM 
    home_sales
WHERE 
    bedrooms = 3 AND 
    bathrooms = 3 AND 
    floors = 2 AND 
    sqft_living >= 2000
GROUP BY 
    YEAR(date_built)
ORDER BY 
    year_built
"""

# Execute the query
result = spark.sql(query)

# Display the results
result.show()


In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
# SQL query to calculate the average price per "view" rating
query = """
SELECT 
    view, 
    ROUND(AVG(price), 2) AS avg_price
FROM 
    home_sales
GROUP BY 
    view
HAVING 
    AVG(price) >= 350000
ORDER BY 
    view DESC
"""

# Execute the query
result = spark.sql(query)

# Display the results
result.show()

# End the timer for the query
end_time = time.time()

# Calculate and display the runtime
runtime = end_time - start_time


print("--- %s seconds ---" % (time.time() - start_time))

--- 7.200241088867188e-05 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

# Verify if the table is cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' table cached? {is_cached}")

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
# SQL query to calculate the average price per "view" rating
query = """
SELECT 
    view, 
    ROUND(AVG(price), 2) AS avg_price
FROM 
    home_sales
GROUP BY 
    view
HAVING 
    AVG(price) >= 350000
ORDER BY 
    view DESC
"""

# Execute the query on the cached data
result_cached = spark.sql(query)

# Display the results
result_cached.show()

# End the timer for the cached query
end_time_cached = time.time()

# Calculate and display the runtime for the cached query
cached_runtime = end_time_cached - start_time_cached


print("--- %s seconds ---" % (time.time() - start_time))


--- 5.459785461425781e-05 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
# Path to save the partitioned Parquet data
output_path = "home_sales_partitioned"

# Write the DataFrame to Parquet format, partitioned by "date_built"
df.write.partitionBy("date_built").parquet(output_path, mode="overwrite")

print(f"Data successfully partitioned by 'date_built' and saved to {output_path}.")

In [ ]:
# 11. Read the parquet formatted data.
# Read the Parquet data
parquet_df = spark.read.parquet(parquet_path)

# Display the schema and a sample of the data
parquet_df.printSchema()
parquet_df.show(5)

In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("home_sales_parquet")

# Verify the table by running a sample query
spark.sql("SELECT * FROM home_sales_parquet LIMIT 5").show()

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
# SQL query to calculate the average price per "view" rating
query = """
SELECT 
    view, 
    ROUND(AVG(price), 2) AS avg_price
FROM 
    home_sales_parquet
GROUP BY 
    view
HAVING 
    AVG(price) >= 350000
ORDER BY 
    view DESC
"""

# Execute the query on the Parquet data
result_parquet = spark.sql(query)

# Display the results
result_parquet.show()

# End the timer for the Parquet query
end_time_parquet = time.time()

# Calculate and display the runtime for the Parquet query
parquet_runtime = end_time_parquet - start_time_parquet



print("--- %s seconds ---" % (time.time() - start_time))

--- 7.104873657226562e-05 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
park.sql("UNCACHE TABLE home_sales")

# Verify if the table is uncached
is_cached = spark.catalog.isCached("home_sales")
if not is_cached:
    print("The 'home_sales' table has been successfully uncached.")
else:
    print("The 'home_sales' table is still cached.")

In [ ]:
# 15. Check if the home_sales is no longer cached

is_cached = spark.catalog.isCached("home_sales")

# Output the caching status
if not is_cached:
    print("The 'home_sales' table is no longer cached.")
else:
    print("The 'home_sales' table is still cached.")
